# Ch.2 Tutorial

In [1]:
import pulp

## 連立一次方程式

In [2]:
problem = pulp.LpProblem('SLE', pulp.LpMaximize)

In [3]:
x = pulp.LpVariable('x', cat='Continuous')
y = pulp.LpVariable('y', cat='Continuous')

In [4]:
problem += (120 * x + 150 * y == 1440)
problem += (x + y == 10)

In [5]:
status = problem.solve()

In [18]:
print(f'Status: {pulp.LpStatus[status]}')
print(f'x = {x.value()}, y = {y.value()}')

Status: Optimal
x = 2.0, y = 8.0


## 線形計画問題

In [19]:
problem = pulp.LpProblem('LP', pulp.LpMaximize)

In [20]:
x = pulp.LpVariable('x', cat='Continuous')
y = pulp.LpVariable('y', cat='Continuous')

In [22]:
problem += (1 * x + 3 * y <= 30)
problem += (2 * x + 1 * y <= 40)

problem += (x >= 0)
problem += (y >= 0)

problem += (x + 2 * y)

In [23]:
status = problem.solve()

In [24]:
print('Status:', pulp.LpStatus[status])
print('x=', x.value(), 'y=', y.value(),
      'obj=', problem.objective.value())

Status: Optimal
x= 18.0 y= 4.0 obj= 26.0


## 規模の大きな線形計画問題(連続緩和)

In [25]:
import pandas as pd

In [26]:
# データの取得
require_df = pd.read_csv('./PyOptBook/2.tutorial/requires.csv')
stock_df = pd.read_csv('./PyOptBook/2.tutorial/stocks.csv')
gain_df = pd.read_csv('./PyOptBook/2.tutorial/gains.csv')

In [27]:
# リストの定義
P = gain_df['p'].tolist()
M = stock_df['m'].tolist()

In [28]:
# 定数の定義

stock = {row.m:row.stock for row in stock_df.itertuples()}
gain = {row.p:row.gain for row in gain_df.itertuples()}
require = {(row.p, row.m):row.require for row in require_df.itertuples()}

In [29]:
# 数理最適化モデルの定義
problem = pulp.LpProblem('LP2', pulp.LpMaximize)

In [30]:
# 変数の定義
x = pulp.LpVariable.dicts('x', P, cat='Continuous')

In [32]:
# 整数式の定義
for p in P:
    problem += (x[p] >= 0)

for m in M:
    problem += (pulp.lpSum([require[p, m] * x[p] for p in P]) <= stock[m])

In [34]:
# 目的関数の定義
problem += (pulp.lpSum([gain[p] * x[p] for p in P]))

In [35]:
# 求解
status = problem.solve()
print('Status:', pulp.LpStatus[status])

Status: Optimal


In [36]:
# 計算結果の表示
for p in P:
    print(p, x[p].value())
    
print('obj=', problem.objective.value())

p1 12.142857
p2 3.5714286
p3 7.4285714
p4 0.0
obj= 80.42857099999999
